# Processing well location data for carbon bombs
*Feb. 9, 2023*

For a story for CBC News, we want to find out which companies are most active in various "carbon bombs" around Canada. We can look at this a variety of ways: how much money the companies have invested in the projects, how many facilities they've built there, or how much land they control in the area.

For this analysis, we'll start with data from the BC Oil and Gas Commission and Alberta's Energy Regulator, and join them with shapefiles for the larger resource play boundaries to show who are the most active companies in each carbon bomb.

We start by bringing in and cleaning the well data. We'll import relevant libraries.

In [321]:
import geopandas
import pandas as pd
import datawrappergraphics

## 1. Which companies are accountable in each project?

We'll start with BC well location data [from the BCOGC](https://data-bcogc.opendata.arcgis.com/datasets/5ace26f614b9435492d679d766430143_0/explore).

In [322]:
bcwells = geopandas.read_file("bc-wells.zip").to_crs("EPSG:4326")

bcwells.head(3)

,OBJECTID,FACILITY_I,FACILITY_T,FACILITY_1,FACILITY_N,STATUS,MAXIMUM_H2,APPLICATIO,ACTIVITY_A,ACTIVITY_C,LEGACY_OGC,PROPONENT,AUTHORITY_,DATA_SOURC,geometry
0,12310952,00026233,B,Battery Site,VERMILION MICA 8-22-81-14 002,Active,0.0,100100214,2016-09-09,None,None,VERMILION ENERGY INC.,OGAA,GPSD,POINT (-120.08995 56.03552)
1,12310953,00026269,WF,Well Facility,TOURMALINE DOE 16-12-080-16 001,Active,700.0,100101140,2016-12-03,None,None,Tourmaline Oil Corp.,OGAA,GPSD,POINT (-120.35160 55.92593)
2,12310954,00026255,WF,Well Facility,ARCRES DAWSON CREEK 03-15-080-15 001,Active,1.5,100100723,2016-12-08,None,None,ARC Resources Ltd.,OGAA,CGWC,POINT (-120.26072 55.92817)


This dataset contains more than just wells, so let's see what facility types are included.

In [323]:
bcwells["FACILITY_1"].unique()

array(['Battery Site', 'Well Facility', 'Satellite Battery',
       'Oil Sales Meter', 'Gas Processing Plant', 'Compressor Station',
       'Pump Station', 'Gas Sales Meter', 'Water Hub', 'Disposal Station',
       'Processing Battery', 'LNG Facility', 'Compressor Dehydrator',
       'Injection Station', 'Tank Terminal', 'Gas Dehydrator',
       'Pipeline Equipment', 'NGL Fractionation Facility',
       'Pipeline Gathering'], dtype=object)

And we'll select only those labeled "Well Facility".

In [324]:
bcwells = bcwells[bcwells["FACILITY_1"] == "Well Facility"]

We only want active wells, so let's take a look at the STATUS column to see what the options are.

In [325]:
bcwells["STATUS"].unique()

array(['Active', 'Suspended', 'Inactive', 'Under Construction',
       'Permit Approved', 'Removed', 'Construction Complete'],
      dtype=object)

From this list, we probably only want "Active" wells.

In [326]:
activebcwells = bcwells[bcwells["STATUS"] == "Active"]

activebcwells.head(3)

,OBJECTID,FACILITY_I,FACILITY_T,FACILITY_1,FACILITY_N,STATUS,MAXIMUM_H2,APPLICATIO,ACTIVITY_A,ACTIVITY_C,LEGACY_OGC,PROPONENT,AUTHORITY_,DATA_SOURC,geometry
1,12310953,00026269,WF,Well Facility,TOURMALINE DOE 16-12-080-16 001,Active,700.0,100101140,2016-12-03,None,None,Tourmaline Oil Corp.,OGAA,GPSD,POINT (-120.35160 55.92593)
2,12310954,00026255,WF,Well Facility,ARCRES DAWSON CREEK 03-15-080-15 001,Active,1.5,100100723,2016-12-08,None,None,ARC Resources Ltd.,OGAA,CGWC,POINT (-120.26072 55.92817)
4,12310956,00026301,WF,Well Facility,ARCRES PARKLAND 04-21-081-17 001,Active,0.0,100101477,2017-03-07,None,None,ARC Resources Ltd.,OGAA,SNK,POINT (-120.60800 56.03237)


We'll ultimately join this data with the Alberta data, so let's put a "province" column on it, in case we need to separate them again later.

In [327]:
activebcwells["PROVINCE"] = "BRITISH COLUMBIA"

c:\Users\dexmc\anaconda3\envs\geo\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


We'll also select just the columns we need, and rename them to something more standard.

In [328]:
activebcwells = activebcwells[["OBJECTID", "PROPONENT", "PROVINCE", "geometry"]]
activebcwells.columns = ["ID", "COMPANY", "PROVINCE", "geometry"]

activebcwells

,ID,COMPANY,PROVINCE,geometry
1,12310953,Tourmaline Oil Corp.,BRITISH COLUMBIA,POINT (-120.35160 55.92593)
2,12310954,ARC Resources Ltd.,BRITISH COLUMBIA,POINT (-120.26072 55.92817)
4,12310956,ARC Resources Ltd.,BRITISH COLUMBIA,POINT (-120.60800 56.03237)
10,12310962,VERMILION ENERGY INC.,BRITISH COLUMBIA,POINT (-120.11768 56.07899)
11,12310963,PETRONAS Energy Canada Ltd.,BRITISH COLUMBIA,POINT (-122.17818 56.97862)
...,...,...,...,...
10746,12321698,Canadian Natural Resources Limited,BRITISH COLUMBIA,POINT (-120.13085 56.70540)
10747,12321699,Pacific Canbriam Energy Limited,BRITISH COLUMBIA,POINT (-122.02956 56.28518)
10748,12321700,Ovintiv Canada ULC,BRITISH COLUMBIA,POINT (-120.23237 55.18030)
10750,12321702,Ovintiv Canada ULC,BRITISH COLUMBIA,POINT (-120.17866 55.19430)


Now let's read in the [Alberta well location data](https://www.aer.ca/providing-information/data-and-reports/statistical-reports/st37).

In [329]:
abwells = geopandas.read_file("ab-wells.zip").to_crs("EPSG:4326")

abwells.head(3)

,Licence,CompName,Latitude,Longitude,KBE,GroundElev,SurfLoc,EDCT,RatingLev,LicStatus,LicStatDat,OpSurvProv,geometry
0,0000001,BP Canada Energy Group ULC(A62D),53.386619,-113.617905,700.4,697.4,08-09-051-25W4,BWL,J,RecExempt,05 Jul 1949,None,MULTIPOINT (-113.61791 53.38662)
1,0000002,Canadian Turbo Inc.(0M69),53.256289,-113.680744,724.5,719.0,05-25-049-26W4,BWL,J,RecExempt,12 Aug 1949,None,MULTIPOINT (-113.68075 53.25629)
2,0000003,BP Canada Energy Group ULC(A62D),54.346682,-113.334918,687.6,683.7,07-11-062-23W4,BWL,J,RecExempt,21 Jul 1949,None,MULTIPOINT (-113.33492 54.34668)


Just like we did for the BC data, we'll see what values exist in the "Licence Status" column.

In [330]:
abwells["LicStatus"].unique()

array(['RecExempt', 'Abandoned', 'Suspension', 'RecCertified', 'Issued',
       'Re-Entered', 'Amended', 'Re-entered'], dtype=object)

There's a capitalization issue with these statuses, so let's put them all to upper case.

In [331]:
abwells["LicStatus"] = abwells["LicStatus"].str.upper()
abwells["LicStatus"].unique()

array(['RECEXEMPT', 'ABANDONED', 'SUSPENSION', 'RECCERTIFIED', 'ISSUED',
       'RE-ENTERED', 'AMENDED'], dtype=object)

Here, Rec is short for "reclamation", which is a process that kicks in when a well is finished and not active. So, from this list, we probably want: Amended, Issued, and Re-Entered.

In [332]:
activeabwells = abwells[abwells["LicStatus"].isin(["AMENDED", "ISSUED", "RE-ENTERED"])]

activeabwells.head(3)

,Licence,CompName,Latitude,Longitude,KBE,GroundElev,SurfLoc,EDCT,RatingLev,LicStatus,LicStatDat,OpSurvProv,geometry
15,0000018,Conifer Energy Inc.(A8CW),53.979859,-113.103006,629.7,627.6,01-06-058-21W4,BWL,J,ISSUED,02 Jun 1949,None,MULTIPOINT (-113.10301 53.97986)
16,0000019,Ohana Resources Inc.(A6N5),53.939874,-113.071762,623.3,620.3,05-21-057-21W4,BWL,J,ISSUED,02 Jun 1949,None,MULTIPOINT (-113.07176 53.93987)
23,0000026,Conifer Energy Inc.(A8CW),53.914384,-113.059492,628.5,625.4,10-09-057-21W4,BWL,J,ISSUED,07 Jun 1949,None,MULTIPOINT (-113.05949 53.91438)


Because these rows sound like they might be licences and not wells, let's take a second to do a quick check. We want wells, and if there are duplicates that represent changes in well status or multiple holes in one well fixture, we want to remove those duplicates.

To do this, we'll show which wells have more than one entry, if any.

In [333]:
test = activeabwells.groupby("Licence").count().loc[:, ["CompName"]]
test = test[test["CompName"] > 1]

test

,CompName
Licence,
0001220,2
0001223,2
0001702,2
0001881,2
0002453,2
...,...
B0001129,3
B0001916,2
B0001917,2


Looks like there are about 9000 of them. Let's remove duplicates based on geometry.

In [334]:
activeabwells_nodupes = activeabwells.drop_duplicates("geometry")
activeabwells_nodupes["PROVINCE"] = "ALBERTA"
activeabwells_nodupes = activeabwells_nodupes[["Licence", "CompName", "PROVINCE", "geometry"]]

activeabwells_nodupes.columns = ["ID", "COMPANY", "PROVINCE", "geometry"]

activeabwells_nodupes.head(3)

c:\Users\dexmc\anaconda3\envs\geo\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,ID,COMPANY,PROVINCE,geometry
15,0000018,Conifer Energy Inc.(A8CW),ALBERTA,MULTIPOINT (-113.10301 53.97986)
16,0000019,Ohana Resources Inc.(A6N5),ALBERTA,MULTIPOINT (-113.07176 53.93987)
23,0000026,Conifer Energy Inc.(A8CW),ALBERTA,MULTIPOINT (-113.05949 53.91438)


Let's double check to make sure they're removed.

In [335]:
activeabwells.shape, activeabwells_nodupes.shape

((196052, 13), (173501, 4))

The Alberta data inputs as Multipoint geometry, even though all the points are single points. Let's fix that and turn them into points.

In [336]:
# activeabwells_nodupes["geometry"] = [Point(pt[0].y, pt[0].x) for pt in activeabwells_nodupes["geometry"]]

In [337]:
activeabwells_nodupes

,ID,COMPANY,PROVINCE,geometry
15,0000018,Conifer Energy Inc.(A8CW),ALBERTA,MULTIPOINT (-113.10301 53.97986)
16,0000019,Ohana Resources Inc.(A6N5),ALBERTA,MULTIPOINT (-113.07176 53.93987)
23,0000026,Conifer Energy Inc.(A8CW),ALBERTA,MULTIPOINT (-113.05949 53.91438)
25,0000028,Conifer Energy Inc.(A8CW),ALBERTA,MULTIPOINT (-113.05950 53.91800)
30,0000033,Ohana Resources Inc.(A6N5),ALBERTA,MULTIPOINT (-113.05345 53.93988)
...,...,...,...,...
500565,X0000484,Domestic Water Well(0FF3),ALBERTA,MULTIPOINT (-110.34194 51.49430)
500566,X0000485,Domestic Water Well(0FF3),ALBERTA,MULTIPOINT (-110.28932 51.51252)
500717,Y0001122,Suncor Energy Inc.(0054),ALBERTA,MULTIPOINT (-111.93481 56.40217)
501729,Y0002240,Suncor Energy Inc.(0054),ALBERTA,MULTIPOINT (-111.86324 57.01915)


Now let's combine both of our provinces into one dataset.

In [338]:
wells = pd.concat([activeabwells_nodupes, activebcwells])
wells["COMPANY"] = wells["COMPANY"].str.replace("\(.*\)", "", regex=True)

wells.head(3)

,ID,COMPANY,PROVINCE,geometry
15,0000018,Conifer Energy Inc.,ALBERTA,MULTIPOINT (-113.10301 53.97986)
16,0000019,Ohana Resources Inc.,ALBERTA,MULTIPOINT (-113.07176 53.93987)
23,0000026,Conifer Energy Inc.,ALBERTA,MULTIPOINT (-113.05949 53.91438)


Now we have our data! From this point onward, we'll combine it with various geospatial datasets to show which companies have the most wells in certain areas.

### Montney, Duvernay, and Wilrich

First, we'll import what I've found to be the best shapefile of the Montney formation. Note that this file is spilt into various sub-polygons for the region, but we just want one boundary, so we'll apply a dissolve as well.

In [339]:
montneyb = geopandas.read_file("montney-boundary.zip").to_crs("EPSG:4326").dissolve("MAP_NAME").reset_index()

montneyb

,MAP_NAME,geometry,ID,DESCR1
0,Montney Isopach and Lithofacies,"POLYGON ((-116.82320 53.20750, -116.88293 53.2...",744.0,"Facies 2: Siltstone, gray, argillaceous and sl..."


Let's also rename the name column to one that will match our next file that we're bringing in.

In [340]:
montneyb = montneyb[["MAP_NAME", "geometry"]].rename(columns={"MAP_NAME": "Shale_Unit"})

montneyb

,Shale_Unit,geometry
0,Montney Isopach and Lithofacies,"POLYGON ((-116.82320 53.20750, -116.88293 53.2..."


Let's simplify the name to "Montney" while we're here.

In [341]:
montneyb.at[0,"Shale_Unit"] = "Montney"

montneyb

,Shale_Unit,geometry
0,Montney,"POLYGON ((-116.82320 53.20750, -116.88293 53.2..."


Now we'll pull in shapefiles for Duvernay and Wilrich (part of the Spirit River formation).

In [342]:
boundaries = geopandas.read_file("boundaries.zip").to_crs("EPSG:4326")

boundaries

,Shale_Unit,Figure_Ref,Report_Ref,geometry
0,Duvernay,Figure 2.1.1,Energy Resources Conservation Board/Alberta Ge...,"MULTIPOLYGON (((-117.00064 53.82230, -116.9987..."
1,Wilrich,Figure 2.6.1,Energy Resources Conservation Board/Alberta Ge...,"POLYGON ((-116.80236 53.37851, -116.83238 53.3..."
2,North Nordegg,Figure 2.5.1,Energy Resources Conservation Board/Alberta Ge...,"POLYGON ((-119.61939 56.92935, -119.60370 56.9..."
3,Montney,Figure 2.3.1,Energy Resources Conservation Board/Alberta Ge...,"MULTIPOLYGON (((-116.31136 52.83475, -116.3178..."
4,Muskwa,Figure 2.2.1,Energy Resources Conservation Board/Alberta Ge...,"POLYGON ((-115.75099 57.10330, -115.77128 57.0..."
5,Basal Banff/Exshaw,Figure 2.4.1,Energy Resources Conservation Board/Alberta Ge...,"POLYGON ((-110.00507 51.87972, -110.00506 51.8..."


We'll filter out columns we don't need from this file as well.

In [343]:
boundaries = boundaries.loc[boundaries["Shale_Unit"] != "Montney", ["Shale_Unit", "geometry"]]

boundaries

,Shale_Unit,geometry
0,Duvernay,"MULTIPOLYGON (((-117.00064 53.82230, -116.9987..."
1,Wilrich,"POLYGON ((-116.80236 53.37851, -116.83238 53.3..."
2,North Nordegg,"POLYGON ((-119.61939 56.92935, -119.60370 56.9..."
4,Muskwa,"POLYGON ((-115.75099 57.10330, -115.77128 57.0..."
5,Basal Banff/Exshaw,"POLYGON ((-110.00507 51.87972, -110.00506 51.8..."


In [344]:
boundaries = pd.concat([boundaries, montneyb])

boundaries

,Shale_Unit,geometry
0,Duvernay,"MULTIPOLYGON (((-117.00064 53.82230, -116.9987..."
1,Wilrich,"POLYGON ((-116.80236 53.37851, -116.83238 53.3..."
2,North Nordegg,"POLYGON ((-119.61939 56.92935, -119.60370 56.9..."
4,Muskwa,"POLYGON ((-115.75099 57.10330, -115.77128 57.0..."
5,Basal Banff/Exshaw,"POLYGON ((-110.00507 51.87972, -110.00506 51.8..."
0,Montney,"POLYGON ((-116.82320 53.20750, -116.88293 53.2..."


Even though we're only really interested in three of these shapes, we'll keep them all for the join so we can classify as many wells as possible, in case we need that info in the future.

Now we join to our main dataset of wells.

In [345]:
joined = geopandas.sjoin(wells, boundaries)
joined = joined[["ID", "COMPANY", "Shale_Unit", "geometry"]]

joined

,ID,COMPANY,Shale_Unit,geometry
52,0000055,"Benson, Keith And Jessie",Basal Banff/Exshaw,MULTIPOINT (-111.73793 50.42805)
56,0000059,Department Of National Defence,Basal Banff/Exshaw,MULTIPOINT (-111.17012 50.25728)
122,0000129,"Petersen, Dennis & Inga",Basal Banff/Exshaw,MULTIPOINT (-111.72454 50.38036)
151,0000158,"Wester, Arne John & Bessie Jean",Basal Banff/Exshaw,MULTIPOINT (-111.68591 50.39644)
201,0000209,Curry Cattle Company,Basal Banff/Exshaw,MULTIPOINT (-111.56529 50.67584)
...,...,...,...,...
480380,0500437,Archer Exploration Corp.,Muskwa,MULTIPOINT (-119.26740 58.48216)
481169,0500795,Archer Exploration Corp.,Muskwa,MULTIPOINT (-119.25736 58.47439)
489233,0504943,2060547 Alberta Ltd.,Muskwa,MULTIPOINT (-118.72592 56.57102)
490568,0505539,Enercapita Energy Ltd.,Muskwa,MULTIPOINT (-119.41513 56.63353)


Now let's filter for only the 3 formations we're interested in for this analysis using a pivot table.

In [346]:
ofinterest = ["Montney", "Duvernay", "Wilrich"]

pivot = joined.pivot_table(columns="Shale_Unit", index="COMPANY", values="ID", aggfunc='count')
pivot["sum"] = pivot.sum(axis=1)
pivot = pivot.sort_values("sum", ascending=False)
pivot = pivot[ofinterest]

pivot

Shale_Unit,Montney,Duvernay,Wilrich
COMPANY,,,
Canadian Natural Resources Limited,5144.0,2884.0,4777.0
Torxen Energy Ltd.,NaN,NaN,NaN
Ipc Canada Ltd.,NaN,67.0,67.0
Tourmaline Oil Corp.,2764.0,1289.0,2285.0
Cenovus Energy Inc.,1947.0,1668.0,1694.0
...,...,...,...
Fort Smith Exploration Ltd.,NaN,1.0,NaN
Petroshale Inc.,NaN,NaN,NaN
Frontier Energy Inc.,NaN,NaN,NaN


### Liard Shale

Let's handle the Liard shale formation, which is in a separate shapefile from the Montney shapefile, but will use a similar methodology.

In [347]:
liard = geopandas.read_file("https://geoweb-ags.bcogc.ca/arcgis/rest/services/ADMIN/UNCONVENTIONAL_PLAY_TRENDS_PY/MapServer/1/query?outFields=*&where=1%3D1&f=geojson|layername=OGRGeoJSON").to_crs("EPSG:4326")
liard

,NAME,SHAPE_Area,OBJECTID,geometry
0,Liard Basin,2.131918e+10,1,"POLYGON ((-122.97220 59.97532, -122.97155 60.0..."
1,Horn River Basin,1.311802e+10,2,"POLYGON ((-122.97155 60.00234, -122.97220 59.9..."
2,Cordova Basin,2.663334e+09,3,"POLYGON ((-120.93233 60.00513, -120.99418 60.0..."
3,Montney Basin,2.660726e+10,4,"POLYGON ((-120.09707 54.96658, -120.00195 54.9..."


You'll notice here there's a shape for the Montney Basin. We're not using this file because this is the BC side of the Montney only, whereas the other shape is the entire basin across BC and Alberta.

Now we'll spatial join our well data onto this shapefile.

In [348]:
liard_join = geopandas.sjoin(wells, liard)

liard_join.head(3)

,ID,COMPANY,PROVINCE,geometry,index_right,NAME,SHAPE_Area,OBJECTID
386243,0398098,Advantage Energy Ltd.,ALBERTA,MULTIPOINT (-119.99881 55.60391),3,Montney Basin,2.660726e+10,4
492394,0506737,Leucrotta Exploration Inc.,ALBERTA,MULTIPOINT (-119.99948 56.07614),3,Montney Basin,2.660726e+10,4
492395,0506738,Leucrotta Exploration Inc.,ALBERTA,MULTIPOINT (-119.99948 56.07623),3,Montney Basin,2.660726e+10,4


And filter for just the Liard Basin.

In [349]:
liard_join = liard_join[liard_join["NAME"] == "Liard Basin"]

liard_join.head(3)

,ID,COMPANY,PROVINCE,geometry,index_right,NAME,SHAPE_Area,OBJECTID
2154,12313106,Bench Creek Resources Ltd.,BRITISH COLUMBIA,POINT (-123.24535 59.48687),0,Liard Basin,2.131918e+10,1
2277,12313229,Bench Creek Resources Ltd.,BRITISH COLUMBIA,POINT (-123.22267 59.78231),0,Liard Basin,2.131918e+10,1
2279,12313231,Bench Creek Resources Ltd.,BRITISH COLUMBIA,POINT (-123.29405 59.47372),0,Liard Basin,2.131918e+10,1


Now we'll combine Liard and Montney etc tables into one.

Start by pivoting our Liard table to match the Montney one.

In [350]:
liard_pivot = liard_join.pivot_table(columns="NAME", index="COMPANY", values="ID", aggfunc="count")

liard_pivot

NAME,Liard Basin
COMPANY,
Bench Creek Resources Ltd.,115
Crescent Point Energy Corp.,7


Then join the two together.

In [351]:
summary = pd.concat([pivot, liard_pivot], axis=1)

summary

,Montney,Duvernay,Wilrich,Liard Basin
COMPANY,,,,
Canadian Natural Resources Limited,5144.0,2884.0,4777.0,NaN
Torxen Energy Ltd.,NaN,NaN,NaN,NaN
Ipc Canada Ltd.,NaN,67.0,67.0,NaN
Tourmaline Oil Corp.,2764.0,1289.0,2285.0,NaN
Cenovus Energy Inc.,1947.0,1668.0,1694.0,NaN
...,...,...,...,...
Fort Smith Exploration Ltd.,NaN,1.0,NaN,NaN
Petroshale Inc.,NaN,NaN,NaN,NaN
Frontier Energy Inc.,NaN,NaN,NaN,NaN


### The oil sands

There's a better way than counting wells in an area to measure the companies that are most active in the oil sands, because the AER provides a dataset showing showing various projects in the area, and who owns each. So we'll use that instead of our well data for the following carbon bombs:
* Horizon Oil Sands Project
* Kearl
* Athabasca Oil Sands Project
* Syncrude Mildred Lake/Aurora
* Christina Lake

We'll start by loading in our projects, which are spread out over several GDB files.

In [352]:
tables = []

for value in ["2a", "2b", "3c", "3d", "4b", "4c", "4e", "42", "43", "47"]:
    table = geopandas.read_file(f"oilsandsprojects/a000000{value}.gdbtable").to_crs("EPSG:4326")
    tables.append(table)

projects = pd.concat(tables)
projects.head(2)

,OSP_NO,OSP_NAME,OS_Area,SHAPE_Leng,Project_Name,Industry_Type,Project_Status,Year,Id,Shape_Length,Shape_Area,geometry,Operator_Name,SHAPE_Length,SHAPE_Area
0,40,Syncrude,Athabasca,52106.616139,Aurora North,Oil Sands Mine,Operating,2009,0.0,52106.616175,1.021148e+08,"MULTIPOLYGON (((-111.68567 57.29345, -111.6860...",NaN,NaN,NaN
1,39,Athabasca Oil Sands Corp.,Athabasca,12964.776158,MacKay River,Commerical in situ,Operating,2009,0.0,12964.776163,7.862400e+06,"MULTIPOLYGON (((-112.07896 56.82784, -112.0789...",NaN,NaN,NaN


Taking a look at this table, it looks like one important column, OSP_NAME, is named something different in one of the datasets: Operator_Name. Let's fix that by moving data from Operator_Name column into OSP_NAME where applicable.

In [353]:
projects.loc[projects["OSP_NAME"].isna(), "OSP_NAME"] = projects.loc[projects["OSP_NAME"].isna(), "Operator_Name"]

projects.head(2)

,OSP_NO,OSP_NAME,OS_Area,SHAPE_Leng,Project_Name,Industry_Type,Project_Status,Year,Id,Shape_Length,Shape_Area,geometry,Operator_Name,SHAPE_Length,SHAPE_Area
0,40,Syncrude,Athabasca,52106.616139,Aurora North,Oil Sands Mine,Operating,2009,0.0,52106.616175,1.021148e+08,"MULTIPOLYGON (((-111.68567 57.29345, -111.6860...",NaN,NaN,NaN
1,39,Athabasca Oil Sands Corp.,Athabasca,12964.776158,MacKay River,Commerical in situ,Operating,2009,0.0,12964.776163,7.862400e+06,"MULTIPOLYGON (((-112.07896 56.82784, -112.0789...",NaN,NaN,NaN


Now we want to filter by project name. We'll use case insensitive regex via the str.contains() method.

Jackpine, Scotford, and Muskeg River are all part of the Athabasca Oilsands Project, which is why they appear in the string below. The others are self explanatory - they're the carbon bomb projects we're interested in.

In [354]:
projectsearchstring = "Horizon|Kearl|Christina Lake|Mildred Lake|Muskeg River|Scotford|Jackpine"

Here we do the filtering.

In [355]:
subset = (projects[projects["Project_Name"].str.contains(projectsearchstring, regex=True, case=False)]
          .drop_duplicates("geometry")
          .sort_values("Project_Name")
          )

subset.head(3)

,OSP_NO,OSP_NAME,OS_Area,SHAPE_Leng,Project_Name,Industry_Type,Project_Status,Year,Id,Shape_Length,Shape_Area,geometry,Operator_Name,SHAPE_Length,SHAPE_Area
47,25,Cenovus FCCL Ltd.,Athabasca,NaN,Christina Lake,Commercial in situ,Operating,2011,NaN,44747.182409,7.694464e+07,"MULTIPOLYGON (((-110.93742 55.58454, -110.9115...",NaN,NaN,NaN
20,25,Cenovus,Athabasca,52014.998004,Christina Lake,Commercial in situ,Operating,2009,0.0,52014.998079,1.136820e+08,"MULTIPOLYGON (((-110.78214 55.63548, -110.7821...",NaN,NaN,NaN
11,25,Encana,ATHA,NaN,Christina Lake,02,01,2007,NaN,NaN,NaN,"MULTIPOLYGON (((-110.78214 55.63548, -110.7821...",Encana,52014.998079,113682035.651151


And now let's group the smaller subprojects into project names that align with our carbon bomb research.

In [356]:
subset.loc[subset["Project_Name"].str.contains("Christina") ,"Project"] = "Christina Lake"
subset.loc[subset["Project_Name"].str.contains("Jackpine|Scotford|Muskeg") ,"Project"] = "Athabasca Oil Sands Project"
subset.loc[subset["Project_Name"].str.contains("Mildred") ,"Project"] = "Mildred Lake"
subset.loc[subset["Project_Name"].str.contains("Horizon") ,"Project"] = "Horizon Mine"
subset.loc[subset["Project_Name"].str.contains("Kearl") ,"Project"] = "Kearl Mine"

subset.head(3)

,OSP_NO,OSP_NAME,OS_Area,SHAPE_Leng,Project_Name,Industry_Type,Project_Status,Year,Id,Shape_Length,Shape_Area,geometry,Operator_Name,SHAPE_Length,SHAPE_Area,Project
47,25,Cenovus FCCL Ltd.,Athabasca,NaN,Christina Lake,Commercial in situ,Operating,2011,NaN,44747.182409,7.694464e+07,"MULTIPOLYGON (((-110.93742 55.58454, -110.9115...",NaN,NaN,NaN,Christina Lake
20,25,Cenovus,Athabasca,52014.998004,Christina Lake,Commercial in situ,Operating,2009,0.0,52014.998079,1.136820e+08,"MULTIPOLYGON (((-110.78214 55.63548, -110.7821...",NaN,NaN,NaN,Christina Lake
11,25,Encana,ATHA,NaN,Christina Lake,02,01,2007,NaN,NaN,NaN,"MULTIPOLYGON (((-110.78214 55.63548, -110.7821...",Encana,52014.998079,113682035.651151,Christina Lake


Now we'll pivot and join together the names of companies in each bigger project category.

In [357]:
oilsands = subset.groupby("Project").agg({"OSP_NAME": lambda x: ', '.join(x.str.strip().unique())})

oilsands

,OSP_NAME
Project,
Athabasca Oil Sands Project,"Shell Canada Limited, Shell Canada Energy"
Christina Lake,"Cenovus FCCL Ltd., Cenovus, Encana, MEG Energy..."
Horizon Mine,"Canadian Natural Resources Limited, Canadian N..."
Kearl Mine,Imperial Oil Resources
Mildred Lake,Syncrude


There are several entries for each of these projects, but each appears to be operating based on the data there, but also on online research, so I'll add that manually in a new column.

In [358]:
oilsands["STATUS"] = "ACTIVE"

oilsands

,OSP_NAME,STATUS
Project,,
Athabasca Oil Sands Project,"Shell Canada Limited, Shell Canada Energy",ACTIVE
Christina Lake,"Cenovus FCCL Ltd., Cenovus, Encana, MEG Energy...",ACTIVE
Horizon Mine,"Canadian Natural Resources Limited, Canadian N...",ACTIVE
Kearl Mine,Imperial Oil Resources,ACTIVE
Mildred Lake,Syncrude,ACTIVE


### Coal mines - Gething Coal Mine, Murray River Coal Mine, Fording River

The coal mines on our list are easy - it's not a data operation, but just a simple research mission. We'll include them in this analysis because the goal is to have all the information we need in a table here.

In [359]:
coalmines = (pd.DataFrame({"PROJECT": ["Gething Coal Mine", "Murray River Coal Mine", "Fording River"],
                           "COMPANY": ["Canadian Kailuan Dehua (CKD) Mines", "HD Mining International Ltd.", "Teck Resources Ltd."],
                           "STATUS": ["UNKNOWN", "INACTIVE", "ACTIVE"]
                           }))

coalmines

,PROJECT,COMPANY,STATUS
0,Gething Coal Mine,Canadian Kailuan Dehua (CKD) Mines,UNKNOWN
1,Murray River Coal Mine,HD Mining International Ltd.,INACTIVE
2,Fording River,Teck Resources Ltd.,ACTIVE


Let's pull together all the data that's just text (coal mines and oil sands projects) into one table.

In [360]:
combined = pd.concat([coalmines.set_index("PROJECT"), oilsands.loc[:, ["OSP_NAME", "STATUS"]].rename(columns={"OSP_NAME": "COMPANY"})])

combined

,COMPANY,STATUS
Gething Coal Mine,Canadian Kailuan Dehua (CKD) Mines,UNKNOWN
Murray River Coal Mine,HD Mining International Ltd.,INACTIVE
Fording River,Teck Resources Ltd.,ACTIVE
Athabasca Oil Sands Project,"Shell Canada Limited, Shell Canada Energy",ACTIVE
Christina Lake,"Cenovus FCCL Ltd., Cenovus, Encana, MEG Energy...",ACTIVE
Horizon Mine,"Canadian Natural Resources Limited, Canadian N...",ACTIVE
Kearl Mine,Imperial Oil Resources,ACTIVE
Mildred Lake,Syncrude,ACTIVE


### The bottom line

Let's bring all the ownership info together into one tidy table. We'll find the top 3 in the well data table and add it to the others.

We copy the table above with string data for company ownership.

In [361]:
top_companies = combined.copy()

top_companies

,COMPANY,STATUS
Gething Coal Mine,Canadian Kailuan Dehua (CKD) Mines,UNKNOWN
Murray River Coal Mine,HD Mining International Ltd.,INACTIVE
Fording River,Teck Resources Ltd.,ACTIVE
Athabasca Oil Sands Project,"Shell Canada Limited, Shell Canada Energy",ACTIVE
Christina Lake,"Cenovus FCCL Ltd., Cenovus, Encana, MEG Energy...",ACTIVE
Horizon Mine,"Canadian Natural Resources Limited, Canadian N...",ACTIVE
Kearl Mine,Imperial Oil Resources,ACTIVE
Mildred Lake,Syncrude,ACTIVE


Then loop through each column in our summary table (the well counts). For each column (each formation), we sort and grab the top 3 companies, then join them together and add a new column to the above summary table.

In [362]:
for company in summary.columns:
    top3 = summary[[company]].sort_values(company, ascending=False).head(3).index.to_list()
    top3 = ", ".join(top3)
    top_companies.loc[company, :] = [top3, "ACTIVE"]
    
top_companies = top_companies.reset_index()
    
top_companies

,index,COMPANY,STATUS
0,Gething Coal Mine,Canadian Kailuan Dehua (CKD) Mines,UNKNOWN
1,Murray River Coal Mine,HD Mining International Ltd.,INACTIVE
2,Fording River,Teck Resources Ltd.,ACTIVE
3,Athabasca Oil Sands Project,"Shell Canada Limited, Shell Canada Energy",ACTIVE
4,Christina Lake,"Cenovus FCCL Ltd., Cenovus, Encana, MEG Energy...",ACTIVE
5,Horizon Mine,"Canadian Natural Resources Limited, Canadian N...",ACTIVE
6,Kearl Mine,Imperial Oil Resources,ACTIVE
7,Mildred Lake,Syncrude,ACTIVE
8,Montney,"Canadian Natural Resources Limited, Tourmaline...",ACTIVE
9,Duvernay,"Ember Resources Inc., Canadian Natural Resourc...",ACTIVE


## 2. Visualizing the data

Let's play around with some ideas for visualizing this data.

We want to accomplish a few things with the viz:
1. Show where each carbon bomb project is located.
2. Show which companies are responsible for the projects.
3. Show how much carbon stands to be released from each project if fully realized.

### Where is each carbon bomb project located?

For the boundary files we have in the above analysis, answering this question is easy. We'll bring them in as-is.

In [363]:
montney = boundaries[["Shale_Unit", "geometry"]].rename(columns={"Shale_Unit": "Project"})

liard_boundary = liard[["NAME", "geometry"]].rename(columns={"NAME": "Project"})

combined_boundaries = pd.concat([montney, liard_boundary])

combined_boundaries

,Project,geometry
0,Duvernay,"MULTIPOLYGON (((-117.00064 53.82230, -116.9987..."
1,Wilrich,"POLYGON ((-116.80236 53.37851, -116.83238 53.3..."
2,North Nordegg,"POLYGON ((-119.61939 56.92935, -119.60370 56.9..."
4,Muskwa,"POLYGON ((-115.75099 57.10330, -115.77128 57.0..."
5,Basal Banff/Exshaw,"POLYGON ((-110.00507 51.87972, -110.00506 51.8..."
0,Montney,"POLYGON ((-116.82320 53.20750, -116.88293 53.2..."
0,Liard Basin,"POLYGON ((-122.97220 59.97532, -122.97155 60.0..."
1,Horn River Basin,"POLYGON ((-122.97155 60.00234, -122.97220 59.9..."
2,Cordova Basin,"POLYGON ((-120.93233 60.00513, -120.99418 60.0..."
3,Montney Basin,"POLYGON ((-120.09707 54.96658, -120.00195 54.9..."


Let's filter out the ones we don't want to map. Note this method of removing the unneeded boundaries will also get rid of the correct duplicate for Montney (we want Montney and not Montney Basin. See the explanation above).

In [364]:
combined_boundaries = combined_boundaries[combined_boundaries["Project"].isin(top_companies["index"])]

combined_boundaries

,Project,geometry
0,Duvernay,"MULTIPOLYGON (((-117.00064 53.82230, -116.9987..."
1,Wilrich,"POLYGON ((-116.80236 53.37851, -116.83238 53.3..."
0,Montney,"POLYGON ((-116.82320 53.20750, -116.88293 53.2..."
0,Liard Basin,"POLYGON ((-122.97220 59.97532, -122.97155 60.0..."


The oil sands projects are also easy - they're polygons that we can map. We'll use the subset table, remove duplicates in case there are overlapping polygons, then dissolve to combine by our larger project categories (recall that the project_name field are individual projects, and some of our carbon bombs are made up of several of these OS projects).

In [365]:
os_boundaries = subset[["Project", "geometry"]].drop_duplicates("geometry").dissolve("Project").reset_index()

os_boundaries

,Project,geometry
0,Athabasca Oil Sands Project,"MULTIPOLYGON (((-113.08696 53.78897, -113.0871..."
1,Christina Lake,"POLYGON ((-110.75626 55.60675, -110.75626 55.6..."
2,Horizon Mine,"POLYGON ((-111.61572 57.45346, -111.61637 57.4..."
3,Kearl Mine,"POLYGON ((-111.00889 57.38091, -111.00889 57.3..."
4,Mildred Lake,"MULTIPOLYGON (((-111.55127 57.03639, -111.5584..."


Now we'll combine our oilsands boundaries with the Montney etc. boundary file.

In [366]:
area_boundaries = pd.concat([os_boundaries, combined_boundaries])

area_boundaries

,Project,geometry
0,Athabasca Oil Sands Project,"MULTIPOLYGON (((-113.08696 53.78897, -113.0871..."
1,Christina Lake,"POLYGON ((-110.75626 55.60675, -110.75626 55.6..."
2,Horizon Mine,"POLYGON ((-111.61572 57.45346, -111.61637 57.4..."
3,Kearl Mine,"POLYGON ((-111.00889 57.38091, -111.00889 57.3..."
4,Mildred Lake,"MULTIPOLYGON (((-111.55127 57.03639, -111.5584..."
0,Duvernay,"MULTIPOLYGON (((-117.00064 53.82230, -116.9987..."
1,Wilrich,"POLYGON ((-116.80236 53.37851, -116.83238 53.3..."
0,Montney,"POLYGON ((-116.82320 53.20750, -116.88293 53.2..."
0,Liard Basin,"POLYGON ((-122.97220 59.97532, -122.97155 60.0..."


Now let's map using the [datawrappergraphics package](https://github.com/dexmcmillan/datawrappergraphics). We'll add a "type" column as per the documentation for this package, because we'll add points to this dataset later. Then we'll add some styling options as per the documentation.

In [367]:
area_boundaries["type"] = "area"
area_boundaries["title"] = area_boundaries["Project"]
area_boundaries["stroke"] = False

Map it!

In [368]:
datawrappergraphics.Map("Zf9Iz").data(area_boundaries).head("Carbon bombs in Canada").publish().show()

INFO:root:SUCCESS: Data added to chart.
INFO:root:SUCCESS: Chart head added.
INFO:root:SUCCESS: Chart published!
c:\Users\dexmc\anaconda3\envs\geo\lib\site-packages\IPython\core\display.py:419: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


### Which countries lead on potential emissions?

Let's do up a little bar chart and a world map to show the potency of carbon bomb projects around the world.

We'll start with the raw data from the researchers (I've removed some columns before import to keep things simple).

In [369]:
carbonbombs = pd.read_csv("carbonbombs.csv")

carbonbombs.head(3)

,Project,Country,Gt CO2,Fuel
0,Upper Zakum,United Arab Emirates,6.440383,OilGas
1,Bu Hasa,United Arab Emirates,4.915779,OilGas
2,Bab,United Arab Emirates,4.197033,OilGas


Let's convert our value column to a float so we can properly pivot on it.

In [370]:
carbonbombs["Gt CO2"] = carbonbombs["Gt CO2"].astype(float)

Now let's pivot. We'll also add a sum column so we can sort on it.

In [371]:
world_pivot = carbonbombs.pivot_table(columns="Fuel", index="Country", values="Gt CO2", aggfunc="sum")
world_pivot["Sum"] = world_pivot.sum(axis=1)
world_pivot = world_pivot.sort_values("Sum", ascending=False).fillna(0.0)

world_pivot.head(3)

Fuel,Coal,OilGas,Sum
Country,,,
China,306.104208,26.813278,332.917486
United States,11.327005,139.726635,151.053640
Russian Federation,33.674948,83.349486,117.024434


Let's do a quick calculation to find out how many more times China's potential emissions are (it's huge, so it's of interest). We'll put this in the datawrapper deck.

In [372]:
canada_china = round(world_pivot.at["China", "Sum"] / world_pivot.at["Canada", "Sum"], 1)

Now we'll push data to datawrapper. Note that I've styled these in datawrapper's web app as well.

In [373]:
(datawrappergraphics.Chart("tD7EO")
 .data(world_pivot.head(10))
 .head("Canada has the world's 7th most potent carbon bombs")
 .deck(f"China's carbon bombs are <b>{canada_china} the potency</b> of Canada's.")
 .publish()
 .show()
 )

INFO:root:SUCCESS: Data added to chart.
INFO:root:SUCCESS: Metadata updated.
INFO:root:SUCCESS: Chart head added.
INFO:root:SUCCESS: Chart deck added.
INFO:root:SUCCESS: Chart published!
c:\Users\dexmc\anaconda3\envs\geo\lib\site-packages\IPython\core\display.py:419: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


The map datawrapper.

In [374]:
(datawrappergraphics.Chart("5lKvI")
 .data(world_pivot)
 .publish()
 .show()
 )

INFO:root:SUCCESS: Data added to chart.
INFO:root:SUCCESS: Metadata updated.
INFO:root:SUCCESS: Chart published!
c:\Users\dexmc\anaconda3\envs\geo\lib\site-packages\IPython\core\display.py:419: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Done!

\-30\-